In [ ]:
# 0518 v1
# 새로운 방식 시도 - 좌표를 따라서 움직이는 게 아니라 마커 위치를 따라 이동
# a번 마커 위치에 도달 판단? 카메라 시야에 a번 마커가 중심에 오는지

#1. 아루코 설정 및 카메라 열기
#2. 순서대로 마커 추적
#3. 목표 마커가 카메라 중심 근처에 오면 다음 마커로 진행


In [ ]:
#!/usr/bin/env python3
import rospy
from std_msgs.msg import Int32MultiArray
from geometry_msgs.msg import Twist

class MarkerFollower:
    def __init__(self):
        rospy.init_node("marker_follower_node")

        self.target_ids = [12, 13, 14, 17, 16, 15]
        self.current_index = 0
        self.center_tolerance = 30

        self.cmd_pub = rospy.Publisher("/cmd_vel", Twist, queue_size=10)
        self.center_sub = rospy.Subscriber("/marker_center", Int32MultiArray, self.center_callback)

        rospy.loginfo("🤖 마커 기반 로봇 제어 노드 시작")

    def center_callback(self, msg):
        if self.current_index >= len(self.target_ids):
            rospy.loginfo("✅ 모든 마커 따라가기 완료")
            self.stop()
            return

        cx, cy, marker_id = msg.data
        target_id = self.target_ids[self.current_index]

        if marker_id != target_id:
            return

        # 화면 중심 기준
        screen_center_x = 640  # 1280x720 카메라 기준
        screen_center_y = 360

        error_x = cx - screen_center_x
        error_y = cy - screen_center_y

        twist = Twist()

        # 중심 도달 여부
        if abs(error_x) < self.center_tolerance and abs(error_y) < self.center_tolerance:
            rospy.loginfo(f"✅ 마커 {target_id} 도달. 다음으로 이동.")
            self.stop()
            rospy.sleep(1.0)
            self.current_index += 1
        else:
            twist.angular.z = -0.002 * error_x
            twist.linear.x = 0.05
            self.cmd_pub.publish(twist)

    def stop(self):
        self.cmd_pub.publish(Twist())

if __name__ == "__main__":
    try:
        MarkerFollower()
        rospy.spin()
    except rospy.ROSInterruptException:
        pass
